In [2]:
!pip install pmdarima

   ---------------------------------------- 0.0/614.7 kB ? eta -:--:--
   ----- ---------------------------------- 92.2/614.7 kB 1.7 MB/s eta 0:00:01
   --------------------------------- ------ 522.2/614.7 kB 6.6 MB/s eta 0:00:01
   ---------------------------------------- 614.7/614.7 kB 4.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.8 MB ? eta -:--:--
   ------ --------------------------------- 0.5/2.8 MB 14.4 MB/s eta 0:00:01
   -------- ------------------------------- 0.6/2.8 MB 9.1 MB/s eta 0:00:01
   ----------- ---------------------------- 0.8/2.8 MB 6.1 MB/s eta 0:00:01
   ----------------- ---------------------- 1.2/2.8 MB 7.7 MB/s eta 0:00:01
   --------------------- ------------------ 1.5/2.8 MB 6.9 MB/s eta 0:00:01
   ------------------------ --------------- 1.7/2.8 MB 6.0 MB/s eta 0:00:01
   ------------------------------- -------- 2.2/2.8 MB 6.9 MB/s eta 0:00:01
   ----------------------------------- ---- 2.5/2.8 MB 6.8 MB/s eta 0:00:01
   ---------

In [19]:
import pandas as pd
from pmdarima import auto_arima
import matplotlib.pyplot as plt

In [20]:
import chardet

with open('01. 데이터프레임_병합(최종).csv', 'rb') as file:
    result = chardet.detect(file.read())
    print(result['encoding'])

EUC-KR


In [21]:
#data=pd.read_csv("01. 데이터프레임_병합(최종).csv")
data = pd.read_csv('01. 데이터프레임_병합(최종).csv', encoding='euc-kr')

In [22]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 39 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Unnamed: 0                    200 non-null    int64  
 1   자치구                           200 non-null    object 
 2   년도                            200 non-null    int64  
 3   전기차(대)                        200 non-null    int64  
 4   자동차등록수(대)                     200 non-null    object 
 5   급속                            200 non-null    int64  
 6   완속                            200 non-null    int64  
 7   재정자주도(군)                      200 non-null    float64
 8   교통문화지수                        200 non-null    float64
 9   부동산업_사업체수                     200 non-null    int64  
 10  전문 과학 및 기술 서비스업_사업체수          200 non-null    int64  
 11  부동산업_종사자수                     200 non-null    int64  
 12  정보통신업_사업체수                    200 non-null    int64  
 13  보건업 및

In [25]:
# 결과를 저장할 데이터프레임 초기화
forecast_results = []

# 자치구 리스트 추출
districts = data['자치구'].unique()

# 변수 리스트 (예측하고자 하는 변수들)
variables = [
    '자동차등록수(대)', '급속', '완속', '재정자주도(군)', '교통문화지수',
    '부동산업_사업체수', '전문 과학 및 기술 서비스업_사업체수', "부동산업_종사자수", "정보통신업_사업체수", 
    "보건업 및 사회복지 서비스업_사업체수", "합계_사업체수",	"전문 과학 및 기술 서비스업_종사자수",	"보건업 및 사회복지 서비스업_종사자수",
    "교육 서비스업_사업체수",	"사업시설 관리 사업 지원 및 임대 서비스업_사업체수", "정보통신업_종사자수",	
    "예술 스포츠 및 여가관련 서비스업_사업체수", "합계_종사자수", "협회 및 단체 수리 및 기타 개인서비스업_종사자수", "전기 가스 증기 및 공기조절 공급업_사업체수", 
    "도매 및 소매업_종사자수", "협회 및 단체 수리 및 기타 개인서비스업_사업체수", "숙박 및 음식점업_사업체수",	
    "사업시설 관리 사업 지원 및 임대 서비스업_종사자수", "예술 스포츠 및 여가관련 서비스업_종사자수", 
    "숙박 및 음식점업_종사자수", "교육 서비스업_종사자수", "금융 및 보험업_사업체수",	"수도 하수 및 폐기물 처리 원료 재생업_종사자수",	
    "건설업_종사자수", "전기 가스 증기 및 공기조절 공급업_종사자수"]

# 숫자 변환 함수 정의 (숫자에 ','가 있을 경우 처리)
def to_float(value):
    try:
        return float(str(value).replace(',', ''))
    except ValueError:
        return None

# 숫자형 변수 전처리
data[variables] = data[variables].applymap(to_float)

# 각 자치구별로 모델 생성 및 예측
for district in districts:
    district_data = data[data['자치구'] == district]
    
    for variable in variables:
        if variable not in district_data.columns:
            continue

        # 연도별 데이터 정리
        ts_data = district_data[['년도', variable]].set_index('년도')
        ts_data = ts_data.dropna()  # 결측값 제거

        # Auto ARIMA 모델 학습
        model = auto_arima(ts_data, seasonal=True, m=1, suppress_warnings=True, start_p=0, d=1, start_q=0, max_p=3, max_q=3, 
                            enforce_stationarity=False)

        # 예측값이 음수가 나오지 않도록 처리
        def ensure_positive_forecast(forecast):
            return [max(0, value) for value in forecast]

        
        # 예측 (2025, 2026년)
        future_years = [2025, 2026]
        forecast = ensure_positive_forecast(model.predict(n_periods=len(future_years)))

        # 결과 저장
        for year, value in zip(future_years, forecast):
            forecast_results.append({
                '자치구': district,
                '년도': year,
                '변수': variable,
                '예측값': value
            })

# 결과를 데이터프레임으로 변환
forecast_df = pd.DataFrame(forecast_results)

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_9292\2550334330.py:27: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data[variables] = data[variables].applymap(to_float)
C:\Users\dyj02\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\dyj02\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
C:\Users\dyj02\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\dyj02\anaconda3\Lib\site-packages

In [34]:
# 새로운 형태로 데이터 정리
forecast_pivot = forecast_df.pivot(index=['자치구', '년도'], columns='변수', values='예측값').reset_index()

In [38]:
forecast_pivot

변수,자치구,년도,건설업_종사자수,교육 서비스업_사업체수,교육 서비스업_종사자수,교통문화지수,금융 및 보험업_사업체수,급속,도매 및 소매업_종사자수,보건업 및 사회복지 서비스업_사업체수,...,전기 가스 증기 및 공기조절 공급업_사업체수,전기 가스 증기 및 공기조절 공급업_종사자수,전문 과학 및 기술 서비스업_사업체수,전문 과학 및 기술 서비스업_종사자수,정보통신업_사업체수,정보통신업_종사자수,합계_사업체수,합계_종사자수,협회 및 단체 수리 및 기타 개인서비스업_사업체수,협회 및 단체 수리 및 기타 개인서비스업_종사자수
0,강남구,2025,52954.000000,5378.000000,33342.000000,78.327143,4184.000000,34.000000,151552.000000,3516.000000,...,70.0,1172.000000,15780.000000,134962.000000,9406.000000,108518.428571,108906.0,802164.000000,5970.0,22154.000000
1,강남구,2026,52954.000000,5378.000000,33342.000000,78.327143,4184.000000,34.000000,151552.000000,3516.000000,...,70.0,1172.000000,15780.000000,134962.000000,9406.000000,114724.857143,108906.0,802164.000000,5970.0,22154.000000
2,강동구,2025,22610.000000,2175.000000,11812.000000,79.161429,374.000000,37.000000,28556.000000,1488.857143,...,17.0,61.000000,1590.000000,9103.285714,1128.000000,3414.000000,42408.0,161149.000000,3387.0,5996.857143
3,강동구,2026,22610.000000,2262.000000,11812.000000,79.161429,374.000000,37.000000,28556.000000,1520.714286,...,17.0,61.000000,1590.000000,9502.571429,1128.000000,3414.000000,42408.0,161149.000000,3387.0,5887.714286
4,강북구,2025,4044.000000,889.000000,6274.000000,78.097143,346.000000,5.000000,12534.000000,871.000000,...,6.0,89.000000,812.000000,1950.000000,569.000000,1212.000000,26250.0,80040.000000,2523.0,4155.000000
5,강북구,2026,4044.000000,889.000000,6274.000000,78.097143,346.000000,5.000000,12534.000000,871.000000,...,6.0,89.000000,812.000000,1950.000000,569.000000,1212.000000,26250.0,80040.000000,2523.0,4109.000000
6,강서구,2025,21032.285714,2633.857143,15201.142857,81.492857,599.714286,15.000000,47191.857143,1786.571429,...,22.0,488.000000,3248.000000,32336.000000,2922.000000,19269.142857,59233.0,303541.428571,4468.0,10088.000000
7,강서구,2026,21994.571429,2739.714286,15372.285714,81.492857,637.428571,15.000000,48757.714286,1817.142857,...,22.0,488.000000,3248.000000,32336.000000,2922.000000,20278.285714,59233.0,312184.857143,4468.0,10088.000000
8,관악구,2025,10608.000000,1582.000000,17960.571429,78.078571,232.000000,0.000000,17132.000000,1286.285714,...,14.0,15.000000,1543.000000,6712.000000,1386.000000,5108.000000,37364.0,127406.000000,3392.0,6040.000000
9,관악구,2026,10608.000000,1582.000000,18318.142857,78.078571,232.000000,0.000000,17132.000000,1300.571429,...,14.0,15.000000,1543.000000,6712.000000,1386.000000,5108.000000,37364.0,127406.000000,3392.0,6040.000000


In [39]:
forecast_pivot.to_csv('forecast_results2.csv', index=False)